<a href="https://colab.research.google.com/github/sajjad-yazdanparast/sentiment_analysis/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from gensim.models import KeyedVectors
import pandas as pd 
import numpy as np 
import nltk 
import spacy
import re
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# def gunzip(file_path,output_path):
#     with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
#         shutil.copyfileobj(f_in, f_out)
# gunzip('/content/drive/My Drive/parto tech/ GoogleNews-vectors-negative300.bin.gz','/content/drive/My Drive/parto tech/word2vec.bin')

In [4]:
model = KeyedVectors.load_word2vec_format(open('/content/drive/My Drive/parto tech/sentiment_analysis/word2vec.bin','rb'), binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
train = pd.read_csv('/content/drive/My Drive/parto tech/sentiment_analysis/train.csv') 
test = pd.read_csv('/content/drive/My Drive/parto tech/sentiment_analysis/test.csv')

In [6]:
def remove_at_and_hashtag (text) :
  text = text.replace('@user',' ')
  return text.replace('#',' ')
  #TODO : use hashtags

In [7]:
# lemmatizing with NLTK lemmatizer

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
from nltk import word_tokenize, pos_tag
from collections import defaultdict


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


lem = WordNetLemmatizer()

def my_lemmatizer (text) :
  tokens = word_tokenize(text)
  lemmatized=[]
  poses = [get_wordnet_pos(tag[0]) for tag in pos_tag(tokens)]
  for i in range (len(tokens)):
    lemma = lem.lemmatize(tokens[i],poses[i])
    lemmatized.append(lemma)
  return ' '.join(lemmatized)


In [8]:
# def my_lemmtizer(text) :
#   nlp = spacy.load('en', disable=['parser', 'ner'])
#   doc = nlp(text)
#   string = " ".join([token.lemma_ for token in doc])
#   return re.sub('-PRON-','they',string)
 

In [9]:
train['tweet'] = train['tweet'].apply(remove_at_and_hashtag)
train['tweet'] = train['tweet'].apply(my_lemmatizer)

In [10]:
def sentence2vec(row,w2v) :
  vector = np.zeros((1,300)) 
  words = row.split()
  count = 0 
  for word in words :
    try :
      vector += w2v[word].reshape((1,300))
      count +=1 
    except KeyError :
      continue 
  if count != 0 :
    return vector/count
  return vector

In [32]:
hated_sentences = train[train['label']==1]['tweet']
positive_sentences = train[train['label']==0]['tweet']
hated_vecs = np.concatenate([sentence2vec(z,model) for z in hated_sentences])
positive_vecs = np.concatenate([sentence2vec(z,model) for z in positive_sentences])
from sklearn.manifold import TSNE
ts = TSNE (2)
hated_vecs = ts.fit_transform(hated_vecs)
positive_vecs = ts.fit_transform(positive_vecs)

In [12]:
# import matplotlib.pyplot as plt 
# for row in positive_vecs :
#   plt.plot(row[0],row[1],marker='o',color='g')
# for row in hated_vecs :
#   plt.plot(row[0],row[1],marker='o',color='r')

In [26]:
# train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [33]:
X = np.concatenate((positive_vecs,hated_vecs))
Y = np.concatenate( (np.zeros(len(positive_vecs)) , np.ones(len(hated_vecs))))

In [34]:
len(positive_vecs) / len(hated_vecs)

13.256021409455842

In [35]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X,Y,random_state=101)


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import *
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
# model_ = LogisticRegression(random_state=101)
model_ = SVC(gamma='auto')
# model_ = GaussianNB()
# model_ = XGBClassifier(n_estimators=100,)
# model_ = RandomForestClassifier(class_weight={1:13.2560 , 0:1})
# model_ = LinearSVC(C=1)
model_.fit(x_train,y_train)
y_pred = model_.predict(x_test)

model_.score(x_test,y_test)
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97      7446
         1.0       0.90      0.13      0.23       545

    accuracy                           0.94      7991
   macro avg       0.92      0.57      0.60      7991
weighted avg       0.94      0.94      0.92      7991

